In [1]:
%pwd

'c:\\Users\\Hp\\Documents\\Wine_Quality_End_To_End\\research'

In [2]:
import os
os.chdir("../")

In [3]:
%pwd

'c:\\Users\\Hp\\Documents\\Wine_Quality_End_To_End'

In [13]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/AnilKumar-06/Wine_Quality_End_To_End.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="AnilKumar-06"
os.environ["MLFLOW_TRACKING_PASSWORD"]="43d58c19c0b2d241dce5cd2cf879940fe4f24c11"

In [14]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    metric_file_name: Path
    all_params: dict
    target_column: str
    mlflow_uri: str

In [17]:
from src.WineQuality.constant import *
from WineQuality.utils.common import read_yaml, create_directories, save_json

ModuleNotFoundError: No module named 'yaml'

In [ ]:
class ConfigurationManager:
    def __init__(self,
                config_filepath = CONFIG_FILE_PATH,
                params_filepath = PARAMS_FILE_PATH,
                schema_filepath = SCHEMA_FILE_PATH
                ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_model_evaluation_config(self) ->ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMNS
        
        create_directories([config.root_dir])
        
        model_evaluation_config = ModelEvaluationConfig(
            root_dir = config.root_dir,
            test_data_path = config.test_data_path,
            model_path = config.model_path,
            metric_file_name = config.metric_file_name,
            all_params = params,
            target_column = schema.name,
            mlflow_uri = "https://dagshub.com/AnilKumar-06/Wine_Quality_End_To_End.mlflow"
        )
        
        return model_evaluation_config

In [7]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

ModuleNotFoundError: No module named 'yaml'

In [ ]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
    
    
    def eval_metrics(self, actual, pred):
        
        rmse = np.sqrt(mean_absolute_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        
        return rmse, mae, r2
    
    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)
        
        x_test = test_data.drop([self.config.target_column], axis = 1)
        y_test = test_data[[self.config.target_column]]
        
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_uri_type_store = urlparse(mlflow.get_artifact_uri()).scheme
        
        with mlflow.start_run(nested=True):
            predicted_val = model.predict(x_test)
            (rmse, mae,r2) = self.eval_metrics(y_test, predicted_val)
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)
            
            mlflow.log_params(self.config.all_params)
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)
            
            if tracking_uri_type_store != "file":
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")
                
            else:
                mlflow.sklearn.log_model(model, "model")

In [12]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
    
except Exception as e:
    raise e

[2023-11-28 22:58:51,012:  INFO:  common : yaml file : config\config.yaml loaded successfully]
[2023-11-28 22:58:51,032:  INFO:  common : yaml file : params.yaml loaded successfully]
[2023-11-28 22:58:51,041:  INFO:  common : yaml file : schema.yaml loaded successfully]
[2023-11-28 22:58:51,043:  INFO:  common : Created directories at: artifacts]
[2023-11-28 22:58:51,046:  INFO:  common : Created directories at: artifacts/model_evaluation]


[2023-11-28 22:58:51,970:  INFO:  common : json file saved at : artifacts\model_evaluation\metrics.json]


Registered model 'ElasticnetModel' already exists. Creating a new version of this model...
2023/11/28 22:59:02 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: ElasticnetModel, version 9
Created version '9' of model 'ElasticnetModel'.
